In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from plateinputs.model import Model as sim

## Extracting paleo-elevation forcings

Before starting to run a `goSPL` model we first have to create a series of input files that will be used during the simulation to better constrain the simulation over time. Specifically we want to get for our `goSPL` mesh the plate tectonic displacements (horizontal plate motion), the rainfall conditions and the vertical component required to move from one time step to the following one.

To get all these informations we will use the `plateinputs` library available from:
+ [here](https://github.com/Geodels/plateInputs.git)  
+ to install the library you will need to look at the `README` file in the github repository


<img src="img/geom.png"
     alt="Geometrical solution"
     style="float: left; margin-right: 10px;" />



Once installed, the library required an input file (an example is provided in this folder: `inputearth5Ma.yml`). All the inputs required to run the `goSPL` global model will be save in a folder defined in the `yml` input under the `gospl:` key. 

There are 4 different types of input files that will be created for `goSPL`:

- `mesh_X_XXMa.npz`: defining the initial elevation mesh used in `goSPL`,
- `rain_X_XXMa.npz`: defining the rainfall for considered period of time for the `goSPL` mesh,
- `plate_X_XXMa.npz`: defining `goSPL` nodes horizontal displacements based on considered plate tectonic model,
- `tecto_X_XXMa.npz`: defining `goSPL` geometric tectonic conditions at given time intervals.


Example of how these files are added to `goSPL` input:

```yaml
domain:
    npdata: '../2-inputgen/gospldata/mesh_9_100Ma'
    
plates:
  - start: -100.e6
    plate: '../2-inputgen/gospldata/plate_9_100Ma'
    upsub: '../2-inputgen/gospldata/tecto_9_100Ma'

climate:
  - start: -100.e6
    map: ['../2-inputgen/gospldata/rain_9_100Ma','r']
```

In [ ]:
# Reading input file
model = sim('inputearth5Ma.yml', verbose=True)

# Running through time
model.runProcesses()

In [ ]:
#!python3 runInput.py -i input1Ma.yml -v

#### Important

By default `plateinputs` will export in the `tecto_X_XXMa.npz` file the uplift/subsidence with the key `t` **and** the corresponding paleo-elevation from the model (here Scotese) with the key `z`. 

+ In cases where you want to run a `goSPL` model constrained with the Scotese paleo-elevation (meaning that every 5 Ma the `goSPL` elevation is forced to match with Scotese one) you do not have to change anything.

+ However generally, we do not want to have such constrained in our model and would rather try to incrementally change the uplift/subsidence forcing based on the computed erosion/deposition. In such cases, you will have to remove the `z` key from the file. This is done in the cell below:

In [ ]:
# Starting simulation time 100 Ma
start = -100
# Ending simulation time 0 Ma
end = 0

# Time step (similar to the one used in goSPL input file)
dt = 5

# We create the time array
time = np.arange(start, end, dt)
time

for k in range(len(time)):
    # Read tectonic file
    file = 'gospldata/tecto_9_'+str(int(-time[k]))+'Ma.npz'
    data = np.load(file)
    # Write new tectonic file
    newfile = 'gospldata/ntecto_9_'+str(int(-time[k]))+'Ma.npz'
    np.savez_compressed(newfile, t=data['t'])


You will therefore modify the input file in `goSPL`:

```yaml
plates:
  - start: -100.e6
    plate: '../2-inputgen/gospldata/plate_9_100Ma'
    upsub: '../2-inputgen/gospldata/ntecto_9_100Ma'
```

## Sea-level

In Scotese paleo-elevation models, the sea-level is implicitly considered (this is the position of 0 m elevation) and is given every 5 Ma. 

Here we create a sea-level curve with small fluctuations to ensure that sediment entering the marine environment are not always deposited in the region between 2 consecutive time steps in the model. 

Sea-level curve in `goSPL` is given as an input file with 2 columns: `time` in years and `sl` in metres.

In [ ]:
# Starting simulation time 100 Ma
start = -100.e6
# Ending simulation time 95 Ma
end = -95.e6

# Time step (similar to the one used in goSPL input file)
dt = 10000.

# We create the time array
time = np.arange(start, end+dt, dt)
time

We will create a sea-level curve that will have a 5 m fluctuation around the 0 m elevation.

The sea-level will be randomly defined between the range ensuring that the mean value over the considered period will be at approximately 0 m and the standard deviation at about 2 m.

In [ ]:
# Create a sea-level curves that goes between -5 and -5 m 
a, b = -5., 5.
# with a mean at 0 m and a standard deviation of 2 m
mu, sigma = 0., 2.

# Define the distribution parameters
dist = stats.truncnorm((a - mu) / sigma, (b - mu) / sigma, loc=mu, scale=sigma)

# Get the sea-level values
sl = dist.rvs(len(time))

We now create a pandas dataframe containing the dataset for the 2 columns

In [ ]:
sl_data = {
        'time':time,
        'sl': sl
    }
df = pd.DataFrame(sl_data)
df

Looks fine! 

Let's save it as a `csv` file that will be read by `goSPL`

In [ ]:
fname = 'gospldata/sealevel.csv'
df.to_csv(fname, index = None, header=False)

This is added to `goSPL` input file like this:
    

```yaml  
sea:
    position: 0.
    curve: 'gospldata/sealevel.csv'
```